# 120 Json Data 추출 
### 한글 폴더명으로 추출

In [6]:
# pip install pymysql
!pip install pandas

In [19]:
import pymysql
import json
import pandas as pd
import numpy as np
from collections import OrderedDict
import os
import datetime as dt

violation_ID = list(range(62, 69 + 1))
violation_object_ID = list(range(70, 132 + 1))
count = 0

In [20]:
class_list={
    '1' : '1 보행자도로 통행 위반',
    '2' : '2 동승자 탑승 위반',
    '3' : '3 안전모 미착용 위반',
    '4' : '4 무단횡단 위반',
    '5' : '5 횡단보도 주행 위반',
    '6' : '6 신호 위반',
    '7' : '7 교차로 통행방법 위반',
    '8' : '8 정지선 위반'
}

pm_list={
    '0' : '오토바이',
    '1' : '자전거',
    '2' : '킥보드',
    '3' : '기타'
}

violation_list = {
    '0' : '정상',
    '1' : '위반'
}

In [21]:
def connectiondb():
    _host = '218.155.74.39'
    _uid = 'root'
    _pwd = 'f1soft@170'
    _db_name = 'aihub_video_73'
    _port = 3306
    try:
        con = pymysql.connect(host=_host, user=_uid, password=_pwd, database=_db_name, port=int(_port), autocommit=True)
        return con
    except Exception as e:
        return get_error_msg(e)

In [22]:
def createDirectory(accident_type, pm, violation, directory = '.\\json120\\nor'): 
    try: 
        class_dir = class_list[f'{str(accident_type)}']
        pm_dir = pm_list[f'{str(pm)}']
        violation_dir = violation_list[f'{str(violation)}']
        if not os.path.exists(directory): 
            os.makedirs(directory) 
        if not os.path.exists(f'{directory}\\{class_dir}'):
            os.makedirs(f'{directory}\\{class_dir}')
        if not os.path.exists(f'{directory}\\{class_dir}\\{pm_dir}'):
            os.makedirs(f'{directory}\\{class_dir}\\{pm_dir}')
        if not os.path.exists(f'{directory}\\{class_dir}\\{pm_dir}\\{violation_dir}'):
            os.makedirs(f'{directory}\\{class_dir}\\{pm_dir}\\{violation_dir}')
    except OSError: 
        print("Error: Failed to create the directory.")
    return f'{directory}\\{class_dir}\\{pm_dir}\\{violation_dir}\\' 

In [23]:
def get_Data(DATA_ID = ''):
    where = ''
    if(DATA_ID != ''):
        where = f"WHERE A.DATA_ID = '{DATA_ID}'"
    sql = f'''
        SELECT 
                A.DATA_ID,
                A.FRAME,
                replace(A.FILENAME, '.mp4', '') AS FILENAME,
                COUNT(*) AS CNT
        FROM `DATA` AS A
            LEFT JOIN META AS B
                ON A.DATA_ID = B.DATA_ID
        {where}
        ORDER BY A.DATA_ID ASC
    '''
    with connectiondb() as con:
        with con.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
    return np.array(result)

In [24]:
def get_Meta(DATA_ID):
    sql = f'''
        SELECT A.META_ID,
            A.LABEL_ID,
            A.LABEL_TYPE,
            IFNULL(A.INFO, '{{"startFrame":0,"endFrame":0,"rectData":null}}') as INFO,
            C.`NAME` AS NAME
        FROM META A
            LEFT OUTER JOIN DATA B
                ON A.DATA_ID=B.DATA_ID
            LEFT JOIN `LABEL` AS C
                ON A.LABEL_ID = C.LABEL_ID
        WHERE A.DATA_ID={DATA_ID} ORDER BY A.META_ID ASC
    '''
    with connectiondb() as con:
        with con.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
    return np.array(result)

In [25]:
def get_Data_Detail(DATA_ID):
    sql = f'''
    SELECT 
        left(D.FILENAME, 7) AS VIDEO_ID,
        D.DURATION,
        DD.*,
        IFNULL(M.INFO, '{{"startFrame":0,"endFrame":0,"rectData":null}}') as INFO
    FROM `DATA` AS D
    LEFT JOIN `DATA_DETAIL` AS DD
        ON D.DATA_ID = DD.DATA_ID
    LEFT JOIN `META` AS M
        ON D.DATA_ID = M.DATA_ID
        AND M.LABEL_TYPE = 'VIDEO_TRIMMING'
    WHERE D.DATA_ID = {DATA_ID}
    ORDER BY D.DATA_ID
    '''
    with connectiondb() as con:
        with con.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
    return np.array(result[0])

In [26]:
def set_meta_list(DATA_ID):
    for Data in get_Data(DATA_ID):
        _data_ID = Data[0]
        _frame = int(Data[1])
        _fileName = Data[2]
        FrameData = dict()
        
        # total_meta(프레임 기준 배열) => frame, annotations, cai_annotations
        # 데이터 내 라벨링 메타
        for meta in get_Meta(_data_ID) :
            _meta_ID = meta[0]
            _label_ID = meta[1]
            _label_type = meta[2]
            _meta_data = json.loads(meta[3])
            _label_name = meta[4]
            
            startFrame=int(_meta_data["startFrame"])
            endFrame=int(_meta_data["endFrame"])
            # 메타데이터의 프레임
            for i in range(startFrame,endFrame + 1):
                # i 는 프레임번호
                annotations = []
                real_index = i - startFrame 
                try:
                    if _label_type == 'VIDEO_BBOX':


                        b_box = _meta_data["rectData"][real_index]
                        # BBOX
                        annotations.append('bbox')
                        bbox = []
                        _x1_y1_x2_y2 = []
                        _x1_y1_x2_y2.append(float(b_box['left']))
                        _x1_y1_x2_y2.append(float(b_box['top']))
                        _x1_y1_x2_y2.append(float(b_box['left']) + int(b_box['width']))
                        _x1_y1_x2_y2.append(float(b_box['top']) + int(b_box['height']))
                        bbox.append(_x1_y1_x2_y2)
                        annotations.append(bbox)

                    elif _label_type == 'VIDEO_SEGMENTATION':
                        segmentation = _meta_data["polyData"][real_index]["segmentation"]
                        #print(_meta_ID, real_index)
                        # SEGMENTATION
                        annotations.append('polygon')
                        polygon = []
                        # polygon 배열 
                        for seg in segmentation:
                            _x_y = []
                            _x_y.append(float(seg['x']))
                            _x_y.append(float(seg['y']))
                            polygon.append(_x_y)
                        annotations.append(polygon)
                    else:
                         # TRIMMING
                        annotations.append('trimming')
                        annotations.append(0)
                    annotations.append(_meta_ID)
                    annotations.append(_label_ID)
                    annotations.append(_label_name)
                    # 프레임별 데이터로 추가
                    if FrameData.get(f'{i}') :
                        FrameData[f'{i}'].append(annotations)
                    else:
                        FrameData[f'{i}'] = []    
                        FrameData[f'{i}'].append(annotations)
                except:
                    if not os.path.isfile(r'C:\Repositories\Python\73\cnt120\json120\log.txt'): 
                        f = open("새파일.txt", 'w')
                        f.close()
                    with open(r'C:\Repositories\Python\73\cnt120\json120\log.txt', 'a') as log_file:
                        log_file.write(f'메타테이블 에러 : DataID : {_data_ID}  AnnotationID : {_meta_ID}\n')
                    print(f'메타테이블 에러 : DataID : {_data_ID}  AnnotationID : {_meta_ID}')

    return dict(OrderedDict(sorted(FrameData.items(), key=lambda t: int(t[0])))), _fileName

In [27]:
def Make_Cai_annotation(_value):
    _cai_annotation = OrderedDict()
    # _cai_annotations 내에 배열
    _cai_subject_id = []
    _cai_object_id = []
    for val in _value:
        # 위반 어노테이션
        if np.isin(int(val[3]), violation_object_ID):
            if val[0] == 'bbox':
                _cai_object_id.append(val[3])
            elif val[0] == 'polygon':
                _cai_subject_id.append(val[3])
        elif np.isin(int(val[3]), violation_ID):
            _cai_annotation["category_id"] = str(int(val[3]) - 61)
        
        if _cai_annotation.get("category_id") == None:
            _cai_annotation["category_id"] = str(0)
        _cai_annotation["subject_id"] = _cai_subject_id
        _cai_annotation["object_id"] = _cai_object_id
    return _cai_annotation

In [28]:
def execute(DataList,used_pass=True):
    global count
    # 데이터 리스트 
    for Data in DataList :
        
        #json_data = []
        _json_dict = OrderedDict()
        DataMeta, fileName = set_meta_list(Data)
        
        # 다양성 정보
        _info = OrderedDict()
        _description = OrderedDict()
        _annotations = OrderedDict()
        data_detail = get_Data_Detail(Data)
        # 이미 파일이 있으면 PASS
        if (data_detail[2] == None) or\
            (os.path.exists(f'.\\json120\\{class_list[data_detail[11]]}\\{pm_list[data_detail[10]]}\\{violation_list[data_detail[9]]}\\{fileName}.json')): 
            continue
        _info["video_id"] = data_detail[0]
        _info["clip_id"] = data_detail[3]
        _info["data_id"] = data_detail[2]
        _info["date"] = data_detail[5]
        _info["device"] = data_detail[4]
        _info["is_scripted"] = data_detail[6]
        _info["time"] = data_detail[7]
        _info["weather"] = data_detail[8]
        
        _description["violation"] = data_detail[9]
        _description["PM"] = data_detail[10]
        _description["violation_type"] = data_detail[11]
        _description["duration"] = float(data_detail[1])
        json_trimming = json.loads(data_detail[12])
        _annotations["startFrame"] = json_trimming["startFrame"]
        _annotations["endFrame"] = json_trimming["endFrame"]
        #_json_detail["Trimming"] = data_detail[12]
        #_json_detail["Trimming"].pop('rectData')
        
        _json_dict["info"] = _info
        _json_dict["description"] = _description
        
        
        _annotations_frame = []
        # 한데이터에 대한 반복문
        for key, value in DataMeta.items():
            # 하위에 추가될 메타 변수명 선언
            _json = OrderedDict()
            _annotation = []
            violation_flag = False
            # 프레임 정보 추가
            _json["frame"] = int(key)
            # 한 프레임에 대한 반복문
            for detail in value :
                # 어노테이션 변수 선언
                _anno_info = OrderedDict()
                
                if detail[0] in ['bbox', 'polygon']:
                    # bbox, polygon, category_id 추가
                    _anno_info["annotation_id"] = detail[2]
                    _anno_info["annotation_type"] = detail[0]
                    _anno_info["annotation_info"] = detail[1]
                    _anno_info["category_id"] = detail[3]
                    _anno_info["category_name"] = detail[4]
                    _annotation.append(_anno_info)
                # else :
                #    violation_flag = True

            _json["frame_annotation"] = _annotation

            #if violation_flag == True:
            #    print('cai')
            #    violation_flag = False
            _cai_annotations = []
            _cai_annotations.append(Make_Cai_annotation(value))
            _json["cai_annotation"] = _cai_annotations
            
            _annotations_frame.append(_json)
            
        _annotations["anno_info"] = _annotations_frame
        _json_dict["annotations"] = _annotations
        # json_data.append(_json_dict)
        with open(f'{createDirectory(data_detail[11], data_detail[10], data_detail[9])}\\{fileName}.json', 'w', encoding='utf-8') as make_file:
            count = count + 1
            json.dump(_json_dict, make_file, ensure_ascii = False, indent="\t")

In [29]:
def get_data_id_accident(accident_type, except_data_id = []):
    except_where = ''
    if len(except_data_id) > 0:
        except_where = f'''AND D.DATA_ID NOT IN ({','.join(map(str, except_data_id))})'''
    sql = f'''
            SELECT D.DATA_ID FROM DATA AS D
    LEFT JOIN DATASET AS DS
        ON D.DATASET_ID = DS.DATASET_ID
            LEFT JOIN `DATA_DETAIL` AS DD
        ON D.DATA_ID = DD.DATA_ID
    

    WHERE D.CONFIRM_STATUS = 3
    AND DD.DEVICE IN (1, 2)
    AND MID(D.FILENAME, 17,1) = '{accident_type}'
    AND DS.TITLE LIKE '%abies%'
    {except_where}
    ''' 
    with connectiondb() as con:
        with con.cursor() as cur:
            cur.execute(sql)
            result = np.array(cur.fetchall()).reshape(-1,)
    return result

In [30]:
# accident_type 에 따른 json 출력
for i in [1,2,3,4,5,6,8]:
    execute(get_data_id_accident(i))
print(f'JSON파일 총 변환량 : {count}')

KeyboardInterrupt: 

In [12]:
execute([90])

## Work : 90
